In [1]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.1 MB/s eta 0:00:00


In [2]:
import pickle
import pandas as pd
import numpy as np
import re
import emoji
import spacy
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
nlp = spacy.load("en_core_web_sm")

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import numpy as np
import pickle

# Redefine the custom EnsembleModel class
class EnsembleModel:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2

    def predict_proba(self, X):
        prob_1 = self.model_1.predict_proba(X)[:, 1]
        prob_2 = self.model_2.predict_proba(X)[:, 1]
        avg_prob = (prob_1 + prob_2) / 2
        return np.vstack([1 - avg_prob, avg_prob]).T

    def predict(self, X):
        return np.where(self.predict_proba(X)[:, 1] > 0.5, 1, 0)

print(" EnsembleModel class defined!")

# Load the saved ensemble model
with open("/content/drive/MyDrive/Python/sandhiya/Text_emoji_label/ensemble_model.pkl", "rb") as model_file:
    loaded_model = pickle.load(model_file)

print(" Ensemble model loaded successfully!")


✅ EnsembleModel class defined!
✅ Ensemble model loaded successfully!


In [4]:
import pickle
import numpy as np

# Load the saved model
model_path = "/content/drive/MyDrive/Python/sandhiya/Text_emoji_label/ensemble_model.pkl"  # Update with correct path
with open(model_path, "rb") as model_file:
    final_model = pickle.load(model_file)

# Load the saved TF-IDF vectorizer
vectorizer_path = "/content/drive/MyDrive/Python/sandhiya/Text_emoji_label/tfidf_vectorizer.pkl"  # Update path
with open(vectorizer_path, "rb") as vec_file:
    vectorizer = pickle.load(vec_file)

# Function to preprocess input text (define this based on your cleaning approach)
def clean_text_spacy(text):
    if pd.isna(text):  # Check for NaN values
        return ""  # Return an empty string for missing values

    text = str(text)  # Convert non-string values to string
    text = emoji.demojize(text)  # Convert emojis to text
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters & numbers
    text = text.lower().strip()  # Convert to lowercase and remove extra spaces
    doc = nlp(text)  # Process text with spaCy
    tokens = [token.lemma_ for token in doc if not token.is_stop]  # Lemmatization & Stopword removal
    return ' '.join(tokens)

# Function to predict user input
def predict_user_input(final_model, vectorizer):
    while True:
        user_text = input("\nEnter a message to classify (or type 'exit' to quit): ")
        if user_text.lower() == "exit":
            print("Goodbye! ")
            break

        # Clean the input text
        cleaned_text = clean_text_spacy(user_text)

        # Convert to TF-IDF
        text_tfidf = vectorizer.transform([cleaned_text])

        # Predict
        prediction = final_model.predict(text_tfidf)[0]

        # Convert numeric prediction to label
        label = "Spam" if prediction == 1 else "Ham"

        print(f"Prediction: {label} ")

# Call function with best model and vectorizer
predict_user_input(final_model, vectorizer)



Enter a message to classify (or type 'exit' to quit): exit
Goodbye! 
